In [35]:
import pandas as pd
import keras
import os
import tensorflow as tf

In [31]:
def load_data_from_csv(csv_path: str):
    ori_dataframe = pd.read_csv(csv_path)
    dataframe = ori_dataframe.copy()
    
    
    class_no = dataframe["class_no"] # get the labeled data of each posture data (1 dimension of int)
    class_name = dataframe["class_name"]# get the labeled data of each posture data (1 dimension of string)
    dataframe.drop(columns=['file_name',"class_no","class_name"], inplace=True) # drop the useles data for training
    
    
    x = dataframe
    y = keras.utils.to_categorical(class_no)
    class_name = class_name.unique()
    
    return x,y,class_name,ori_dataframe

In [38]:
x_train,y_train,class_names,_ = load_data_from_csv(r"C:\Users\User\Desktop\sophomore_2nd_semester\Artificial Intelligence\final_project\Human-pose-estimation\code\train_model\test_data.csv")


In [34]:
x_test,y_test,_,test_dataframe= load_data_from_csv(r"C:\Users\User\Desktop\sophomore_2nd_semester\Artificial Intelligence\final_project\Human-pose-estimation\code\train_model\test_data.csv")

In [40]:
from data import BodyPart
import movenet

In [ ]:

def get_center_point(landmarks, left_bodypart, right_bodypart):
    """Calculates the center point of the two given landmarks."""

    left = tf.gather(landmarks, left_bodypart.value, axis=1)
    right = tf.gather(landmarks, right_bodypart.value, axis=1)
    center = (left+right)/2
    return center


ModuleNotFoundError: No module named 'data'